In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib

import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import skew
from scipy.stats import norm
from scipy.stats.stats import pearsonr
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
%matplotlib inline
import sys
sys.path.append("/content/drive/MyDrive/MIDA2/IntelligentMobilityProject/Code")
np.set_printoptions(precision=5, suppress=True)

In [27]:
ds_path = '/content/drive/MyDrive/MIDA2/IntelligentMobilityProject/Survey+dataset/4_DNA_values.csv'

df_DNA = pd.read_csv(ds_path)

In [28]:
df_DNA

,InternetUsers,Concern_environmental_impacts,grouped_Age,Would_subsribe_car_sharing_if_available,Preference_tolls_or_traffic_limitation,Gender
0,0.758621,0.5,0.59,0.333333,0.00,0.0
1,0.689655,0.8,0.11,0.333333,0.25,1.0
2,0.551724,0.6,0.59,0.166667,0.00,1.0
3,0.931034,0.8,1.00,0.000000,0.50,1.0
4,0.327586,0.8,0.35,0.333333,0.00,1.0
...,...,...,...,...,...,...
26600,0.551724,0.3,0.18,0.666667,0.50,0.0
26601,0.551724,0.5,0.26,0.000000,0.00,0.0
26602,0.551724,0.7,0.26,0.333333,0.00,1.0
26603,0.551724,0.7,0.35,0.666667,0.25,1.0


# Clustering


## Distance metrics
The implementation of the distance metrics are taken from the library **_distython_**: [https://github.com/KacperKubara/distython](https://github.com/KacperKubara/distython)


In [16]:
!pip install distython

### Euclidean-Overlap Metric (HEOM)

HEOM is a distance metric taken from [this paper](https://arxiv.org/pdf/cs/9701101.pdf).

HEOM handles categorical, numerical and missing data

$$
d_a(x,y) = 
\begin{equation}
  \begin{cases}
    1, \text{if $x$ or $y$ unknown, else}\\
    overlap(x,y), \text{if $a$ is nominal, else}\\
    normalized\_euclidean\_dist_a(x,y)
  \end{cases}
\end{equation}
$$

$$
overlap(x,y) = 
\begin{equation}
  \begin{cases}
    1, \text{ if x=y}\\
    0, \text{ else}
  \end{cases}
\end{equation}
$$

$$
normalized\_euclidean\_dist_a(x,y) = \frac{|x-y|}{range_a}\\
range_a = max_a - min_a
$$

$$HEOM(x,y) = \sqrt{\sum_{a=1}^m d_a(x_a, y_a)^2}$$
where:
- $x, y$ are datapoints
- $a$ is a feature


In [17]:
from distython import HEOM
heom = HEOM(df_DNA.iloc[:,:], [2,3,4,5])

### Gower

In [18]:
!git clone https://github.com/adrinjalali/scikit-learn.git

fatal: destination path 'scikit-learn' already exists and is not an empty directory.


In [19]:
% cd /content/scikit-learn/

/content/scikit-learn


In [20]:
!git submodule update --init

In [ ]:
!python3 setup.py build_ext --inplace

In [39]:
!pip install gower

In [41]:
import gower
distance_matrix = gower.gower_matrix(df, cat_features=[False, False, False, True, True, True])
distance_matrix

array([[0.     , 0.47483, 0.38448, ..., 0.28948, 0.60782, 0.58115],
       [0.47483, 0.     , 0.46966, ..., 0.23132, 0.24632, 0.43966],
       [0.38448, 0.46966, 0.     , ..., 0.23833, 0.39   , 0.19667],
       ...,
       [0.28948, 0.23132, 0.23833, ..., 0.     , 0.34833, 0.375  ],
       [0.60782, 0.24632, 0.39   , ..., 0.34833, 0.     , 0.36   ],
       [0.58115, 0.43966, 0.19667, ..., 0.375  , 0.36   , 0.     ]],
      dtype=float32)

In [42]:
np.save("/content/drive/MyDrive/MIDA2/IntelligentMobilityProject/Code/Data/distance_matrix_gower.npy", distance_matrix)

In [43]:
distance_matrix.shape

(26605, 26605)

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster, inconsistent
from scipy.spatial.distance import cdist, pdist

HC_base_path = "/content/drive/MyDrive/MIDA2/IntelligentMobilityProject/Code/Data/HC/"
algo = "HC"
sep = "_"
metric = "Gower"
file_type = ".npy"
linkages = ["single", "complete", "average", "weighted", "ward", "centroid", "median"]
for link in linkages:
  HC_path = HC_base_path + metric + '/' + algo + sep + metric + sep + link + file_type

  Z = {}
  Z[metric] = linkage(distance_matrix, link, metric="precomputed")
  np.save(HC_path, Z[metric], allow_pickle=False)